In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import tensorflow as tf

In [5]:
df = pd.read_csv('match_dataframe.csv')
df = df.drop(df.columns[[0]],axis = 1)
df.dtypes


0.assists                      int64
0.baronKills                   int64
0.basicPings                 float64
0.bountyLevel                  int64
0.challenges.abilityUses     float64
                              ...   
9.visionScore                  int64
9.visionWardsBoughtInGame      int64
9.wardsKilled                  int64
9.wardsPlaced                  int64
teamWon                        int64
Length: 1191, dtype: object

In [6]:
df = df.astype(np.float32)
df.dtypes

0.assists                    float32
0.baronKills                 float32
0.basicPings                 float32
0.bountyLevel                float32
0.challenges.abilityUses     float32
                              ...   
9.visionScore                float32
9.visionWardsBoughtInGame    float32
9.wardsKilled                float32
9.wardsPlaced                float32
teamWon                      float32
Length: 1191, dtype: object

Split Data:

Features:

In [7]:
train_data = df.sample(frac=.66,random_state=0)
test_data = df.drop(train_data.index)

Labels:

In [8]:
train_label = train_data.pop('teamWon')
test_label = test_data.pop('teamWon')

Normalize data:

In [9]:
train_stats = train_data.describe().transpose()
train_stats.head()

,count,mean,std,min,25%,50%,75%,max
0.assists,1323.0,5.144369,4.661088,0.0,1.0,4.0,7.0,31.0
0.baronKills,1323.0,0.029478,0.169207,0.0,0.0,0.0,0.0,1.0
0.basicPings,1323.0,11.666667,14.351699,0.0,2.0,7.0,16.0,102.0
0.bountyLevel,1323.0,1.142857,2.277629,0.0,0.0,0.0,1.0,18.0
0.challenges.abilityUses,1323.0,190.984879,108.719582,0.0,117.5,170.0,247.5,976.0


In [10]:
from sklearn.preprocessing import normalize

norm_test_data = normalize(test_data)
norm_train_data = normalize(train_data)

Build Model:

In [11]:
from tensorflow import keras
from tensorflow.keras import layers

model = tf.keras.models.Sequential([
    layers.Dense(128, activation = 'relu',input_shape = [len(train_data.keys())]),
    layers.Dense(128, activation = 'relu'),
    layers.Dense(1,activation='sigmoid')
])
model.compile(optimizer = 'adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               152448    
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 169,089
Trainable params: 169,089
Non-trainable params: 0
_________________________________________________________________


Train Model:

If Verbose = 0, use:

In [13]:
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs):
        if epoch % 100 == 0: print('')
        print('UwU ',end='')

In [14]:
res = model.fit(
    train_data, train_label,
    epochs = 50,
    validation_split = 0.2,
    verbose=0,
    callbacks = [PrintDot()]
)


UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU UwU 

In [15]:
hist = pd.DataFrame(res.history)
hist['epoch'] = res.epoch
hist.tail()

,loss,accuracy,val_loss,val_accuracy,epoch
45,26.949442,0.971645,218.211166,0.898113,45
46,33.352173,0.967864,305.493347,0.867925,46
47,24.551270,0.972590,87.810455,0.935849,47
48,19.821609,0.970699,130.380814,0.909434,48
49,27.305653,0.980151,163.375092,0.920755,49


In [16]:
ev = model.evaluate(test_data,test_label)
print('Model accuracy: ', ev[1])

22/22 [==============================] - 0s 1ms/step - loss: 135.4202 - accuracy: 0.9148
Model accuracy:  0.9148311018943787


In [17]:
model.save('./model/model.h5')

In [18]:
import tensorflowjs as tfjs
# tfjs.converters.save_keras_model(model, 'E:\Programming\WebDev\Porobability\porobability_server/tfModel/')